# VectorDB, Hugging Face & Ollama – Assignment Solutions


## Question 1 Answer
Detailed explanation of VectorDB vs Traditional DBs.

## Question 2 Answer
Types of VectorDBs and their use cases.

## Question 3 Answer
Why ChromaDB is important and its features.

## Question 4 Answer
Benefits of Hugging Face Hub.

## Question 5 Answer
Process & advantages of using pre-trained models.

In [ ]:
# Question 6: ChromaDB setup and sample vector
!pip install chromadb

from chromadb import Client
from chromadb.config import Settings

client = Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory="my_chroma_db"))
collection = client.create_collection("sample_collection")

collection.add(
    ids=["1","2","3"],
    embeddings=[[0.1,0.2,0.3],[0.9,0.8,0.7],[0.4,0.4,0.5]],
    documents=["apple fruit", "car engine", "banana yellow"]
)

results = collection.query(query_embeddings=[[0.12,0.18,0.33]], n_results=2)
results

In [ ]:
# Question 7: Fine-tuning Hugging Face model
!pip install transformers datasets accelerate

from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

tokenized = dataset.map(tokenize, batched=True)

training_args = TrainingArguments(output_dir="./fine_tuned_gpt2", num_train_epochs=1)
trainer = Trainer(model=model, args=training_args, train_dataset=tokenized["train"])
# trainer.train()  # commented to avoid long runtime

In [ ]:
# Question 8: Custom LLM using Ollama
# Commands to run in terminal:
# curl -fsSL https://ollama.com/install.sh | sh
# Create Modelfile and run:
# ollama create myllama -f Modelfile
# ollama run myllama "Explain AI"

In [ ]:
# Question 9: Basic RAG using Ollama + ChromaDB
import chromadb, subprocess
from chromadb.config import Settings

client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory="rag_db"))
collection = client.create_collection("rag_docs")
collection.add(ids=["1"], documents=["Machine learning allows computers to learn patterns."])

query = "What is machine learning?"
results = collection.query(query_texts=[query], n_results=1)
context = results["documents"][0][0]

prompt = f"Use the context to answer:\n{context}\n\nQuestion: {query}"
# result = subprocess.run(["ollama", "run", "llama3"], input=prompt, text=True, capture_output=True)
# print(result.stdout)

In [ ]:
# Question 10: Health-tech chatbot architecture with HF + VectorDB + Ollama
from transformers import AutoTokenizer, AutoModel
import chromadb

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-bert")
model = AutoModel.from_pretrained("d4data/biomedical-bert")

def get_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    output = model(**tokens)
    return output.last_hidden_state.mean(dim=1).detach().numpy().tolist()[0]

client = chromadb.Client()
collection = client.create_collection("medical_docs")
collection.add(ids=["1"], documents=["Jaundice is caused by bilirubin buildup."])

query = "What causes jaundice?"
results = collection.query(query_texts=[query], n_results=1)
results